Libraries

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import shapiro
from scipy.stats import levene
import statistics
import pingouin as pg
from scipy.stats import friedmanchisquare
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
filepath = "/home/friction_15/lab/dissertation/experiment/n_back_separate/" #path to the logfile; every tACS conditions
filepaths = os.listdir(filepath)
trials = range(0, 200)
order = [1, 2, 0]
filepaths = [filepaths[i] for i in order]
# outlier = []
outlier = ['SUBF4P4052']
rt_all = []
accuracy_all = []
hit_length = {k:[] for k in filepaths}
acc_dict = {k:[] for k in filepaths}
dprime_dict = dict()
hit_rate_dict =dict()
fa_rate_dict = dict()
timeDurationDf = pd.DataFrame()
for file in filepaths:
    # print(file)
    files = os.listdir(filepath + file + "/")
    sub_df = pd.DataFrame(index = trials)
    rt_cond = []
    subject_name = []
    accuracy_cond = []
    timeDuration = []
    files_except_outliers = [i for i in files if i[:10] not in outlier]
    dprime_df = pd.DataFrame(index = ['dprime'])
    # print(file)
    fa_rate_df = pd.DataFrame(index = ['falserate'])
    hit_rate_df = pd.DataFrame(index = ['hitrate'])
    for i in files_except_outliers:
        filename = "sub_" + i[8:10]
        df = pd.read_csv(filepath + file +'/' + i, skiprows=3 , delimiter = "\t")
        df1 = df.drop(df.dropna(subset=['Time']).index).reset_index()
        df1.dropna(axis = 1, how = 'all', inplace = True)
        df1.columns = df1.iloc[0]
        df1.drop(index = 0, inplace = True)
        df2 = df1.drop(df1.iloc[:, [0,1]], axis = 1)
        
        df2["RT"] = df2["RT"].astype(float)/10 # convert string to float and dividing it by 10
        df2 = df2[df2["Code"].str.len()>5]
        df2[["TRIAL TYPE", "BLOCK", "STIM", "BACK", "TRIAL NUMBER", "STIM NAME", "STIM IDX", "tCOND", "tRESP"]] = df2["Code"].str.split(';',expand=True)
        df2 = df2.drop(df2.iloc[:, [0]], axis = 1)
        
        # removing practice trials - block == 1
        df2["BLOCK"] = df2["BLOCK"].astype(float)
        df2 = df2[df2['BLOCK'] == 1]
        cols = ["BLOCK", "TRIAL NUMBER", "tCOND", "tRESP"]
        for col in cols:
            temp = df2[col] # strip removes white spaces
            df2.drop(labels=[col], axis=1, inplace= True)
            df2.insert(1, col, temp)

        time = df2["Time"].tolist()
        timeDuration.append(int(time[-1])/60/10000)
        # # reaction time 
        df_rt = df2[df2["tRESP"] == "Hit"]
        df_miss = df2[df2['tRESP'] == "Miss"]
        df_CR = df2[df2['tRESP'] == "Correct Rejection"]
        df_fa = df2[df2['tRESP'] == "False Alarm"]
        # print(df_rt["RT"].mean())
        rt_cond.append(df_rt["RT"].mean())
        subject_name.append(filename)

        accuracy_cond.append((len(df_rt)+len(df_CR))/200*100) 

        hit_length[file].append(len(df_rt))
        # print(filename, len(df_rt))
        acc_dict[file] = accuracy_cond
        
        #d' - dprime measure
        hit_rate =len(df_rt)/(len(df_rt)+len(df_miss))
        fa_rate = round(len(df_fa)/(len(df_fa)+ len(df_CR)),6)

        # print(f'before = {fa_rate}')
        if hit_rate == 1:
            hit_rate = 0.99
        if hit_rate == 0:
            hit_rate = 0.01 
        if fa_rate == 1:
            fa_rate = 0.9967
        if fa_rate == 0:
            fa_rate = 0.0033
        # print(f'after = {fa_rate}')

        dprime_df[filename] = norm.ppf(hit_rate)-norm.ppf(fa_rate)

        fa_rate_df[filename] = fa_rate
        hit_rate_df[filename] = hit_rate
        
        ######## normality test
        # n,p = shapiro(df_rt["RT"])
        # if p > 0.05:
        #     print(filename, n, p)

    hit_rate_dict[file] = hit_rate_df
    fa_rate_dict[file] = fa_rate_df
    rt_all.append(rt_cond)
    accuracy_all.append(accuracy_cond) 
    timeDurationDf[file] = timeDuration 
    # dprime_dict[file] = dprime_df.T     

creating data frames

In [ ]:
dataRt = pd.DataFrame(index=subject_name)
df_er = pd.DataFrame(index=subject_name)

for ind,i in enumerate(filepaths):
    dataRt[i] = rt_all[ind]
   
dataRt.sort_index(inplace=True)

In [ ]:
dprime_dictt = dict()
for key in fa_rate_dict.keys():
    
    dprime_dictt[key] = pd.DataFrame({'d_prime':(norm.ppf(hit_rate_dict[key]) - norm.ppf(fa_rate_dict[key]))[0]}, index=fa_rate_df.T.index)

total stimulation duration

In [ ]:
for i, row in timeDurationDf.T.iterrows():
    print(i + " mean:" + str(row.mean()))
    print(i + " std:" + str(row.std()))
    print(i + " min:" + str(row.min()))
    print(i + " max:" + str(row.max()))

Descriptive statistics

In [ ]:
#Total hits across conditions
hit_conds = pd.DataFrame()
hit_conds['Sham'] = hit_length['Sham']
hit_conds['Fixed dose'] = hit_length['Fixed dose']
hit_conds['Individualised dose'] = hit_length['Individualised dose']

In [ ]:
for key in hit_conds:
    print('Mean')
    print(key, round(hit_conds[key].mean(),3))
    print('STD')
    print(key, round(hit_conds[key].std(),2))
    print('Variance')
    print(key, round(hit_conds[key].var(),2))

dprime mean

In [ ]:
for key in dprime_dict:
    print(key, round(dprime_dict[key].mean(),3))
    print(key, round(dprime_dict[key].std(),2))
    print(key, round(dprime_dict[key].var(),2))

Ploting distribution of hits across conditions

In [ ]:
## no of hits distribution
plt.hist(hit_length["Fixed dose"], bins=5)
plt.xlim(15, 55)
plt.ylim(0, 15)
plt.xlabel("HITS")
plt.ylabel("No of Subjects")

plt.figure()
plt.hist(hit_length["Individualised dose"], bins=5)
plt.xlim(15, 55)
plt.ylim(0, 15)
plt.xlabel("HITS")
plt.ylabel("No of Subjects")

plt.figure()
plt.hist(hit_length["Sham"], bins=5)
plt.xlim(15, 55)
plt.ylim(0, 15)
plt.xlabel("HITS")
plt.ylabel("No of Subjects")


In [ ]:
#accuracy
for key in acc_dict:
    print(key, statistics.mean(acc_dict[key]))
    print(key, statistics.stdev(acc_dict[key]))

normality- condition wise

In [ ]:
#RT
for key in dataRt:
    test, p = shapiro(dataRt[key])
    print(key, test, p)

In [ ]:
#accuracy
for key in acc_dict:
    test, p = shapiro(acc_dict[key])
    print(key, test, p)

In [ ]:
#d_prime - normality and levene test
print("normality test")
for key in dprime_dictt:
    test, p = shapiro(dprime_dictt[key])
    print(key, test, p)

print("levene test")
l,pp = levene(dprime_dictt["Fixed dose"]["d_prime"].to_numpy(), dprime_dictt["Individualised dose"]["d_prime"].to_numpy(), 
                    dprime_dictt["Sham"]["d_prime"].to_numpy())
print( l, pp)
    

one way anova   

In [ ]:
#RT -parametric friedman test anova
friedmanchisquare(dataRt["Sham"], dataRt["Fixed dose"], dataRt["Individualised dose"])

In [ ]:
#accuracy -parametric friedman test anova
friedmanchisquare(acc_dict["Sham"], acc_dict["Fixed dose"], acc_dict["Individualised dose"])

In [ ]:
## d-prime - one way repeated measures anova
#creating the data frame
dprime_df_cond = pd.DataFrame()
dprime_df_cond['Sham'] = dprime_dictt['Sham']
dprime_df_cond['Fixed dose'] = dprime_dictt['Fixed dose']
dprime_df_cond['Individualised dose'] = dprime_dictt['Individualised dose']


In [ ]:
#anova
aov_dprime = pg.rm_anova(data = dprime_df_cond)
aov_dprime

Descriptive statistics

In [ ]:
#Reaction time
for key in dataRt:
    print(key, "mean: ", dataRt[key].mean())
    print("std: ", dataRt[key].std())
    print("var: ", dataRt[key].var())

accuracy    

In [ ]:
#Accuracy
for key in acc_dict:
    print(key, "mean: ", round(statistics.mean(acc_dict[key]), 3))
    print("std: ", round(statistics.stdev(acc_dict[key]),2))
    print("var: ", round(statistics.variance(acc_dict[key]),2))


dprime

In [ ]:
#d-Prime
for key in dprime_dictt:
    print(key)
    print("std: " + str(round(statistics.stdev(dprime_dictt[key]['d_prime'].to_numpy()), 3)))
    print("mean: " + str(round(statistics.mean(dprime_dictt[key]['d_prime'].to_numpy()), 3)))
    print("var: " + str(round(statistics.variance(dprime_dictt[key]['d_prime'].to_numpy()), 3)))

Violin plots

defining functions

In [ ]:
def adjacent_values(vals, q1, q3):
    upper_adjacent_value = q3 + (q3 - q1) * 1.5
    upper_adjacent_value = np.clip(upper_adjacent_value, q3, vals[-1])

    lower_adjacent_value = q1 - (q3 - q1) * 1.5
    lower_adjacent_value = np.clip(lower_adjacent_value, vals[0], q1)
    return lower_adjacent_value, upper_adjacent_value


def set_axis_style(ax, labels):
    ax.get_xaxis().set_tick_params(direction='out')
    ax.xaxis.set_ticks_position('bottom')
    ax.set_xticks(np.arange(1, len(labels) + 1))
    ax.set_xticklabels(labels, fontsize = 18, fontname = "sans")
    ax.set_xlim(0.25, len(labels) + 0.75)
    ax.set_xlabel('Sample name')

rt across cond

In [ ]:
data_to_plot = [dataRt['Sham'], dataRt['Fixed dose'], dataRt['Individualised dose'] ]
fig = plt.figure(figsize=(12, 7))

# Create an axes instance
ax = fig.add_axes([0.09,0.1,0.85,0.84])
ax.tick_params(axis='y', which='major', labelsize=12)
ax.spines.right.set_visible(False)
ax.spines.top.set_visible(False)
ax.spines.left.set_visible(False)
ax.spines.bottom.set_visible(False)
# ax.set_yticks(size = 14)
# ax.set_xticks(size = 16)
ax.grid(True, axis = "y", color='w', linestyle='-', linewidth=1.2, which="major",  zorder = -1.0)
ax.patch.set_facecolor('.91')
plt.rc("axes", axisbelow = True)
# Create the boxplot

bp = ax.violinplot(data_to_plot, showmeans=False, showmedians=False, showextrema=True)
label = ['Sham','Fixed Dose', 'Individualised dose']

for ax in [ax]:
    set_axis_style(ax, label)


ax.set_xlabel('Stimulation condition', fontsize = 22, fontname = "sans")
ax.set_ylabel('RT (ms)', fontsize = 18, fontname = "sans")
ax.set_title('Reaction time', fontsize = 26, fontname = "sans")

plt.rc('xtick', labelsize=30) 
plt.rc('ytick', labelsize=15) 
plt.rc('axes', titlesize=35)   
plt.rc('axes', labelsize=35)  

for pc in bp['bodies']:
    pc.set_facecolor('#B2C8DF')
    pc.set_edgecolor('#6E85B7')
    pc.set_alpha(0.7)
quartile1, medians, quartile3 = np.percentile(data_to_plot, [25, 50, 75], axis=1)
whiskers = np.array([
    adjacent_values(sorted_array, q1, q3)
    for sorted_array, q1, q3 in zip(data_to_plot, quartile1, quartile3)])
whiskers_min, whiskers_max = whiskers[:, 0], whiskers[:, 0]

inds = np.arange(1, len(medians) + 1)
ax.scatter(inds, medians, marker='o', color='black', s=30, zorder=3)
ax.vlines(inds, quartile1, quartile3, color='black', linestyle='-', lw=5)
ax.vlines(inds, whiskers_min, whiskers_max, color='black', linestyle='-', lw=4)
# plt.savefig("/home/friction_15/lab/dissertation/reviewFigures/violinTacsRT.jpg", format="jpg", dpi = 500)




Accuracy

In [ ]:
data_to_plot = [acc_dict['Sham'], acc_dict['Fixed dose'], acc_dict['Individualised dose'] ]
fig = plt.figure(figsize=(12, 7))

# Create an axes instance
ax = fig.add_axes([0.09,0.1,0.85,0.84])
ax.tick_params(axis='y', which='major', labelsize=12)
ax.spines.right.set_visible(False)
ax.spines.top.set_visible(False)
ax.spines.left.set_visible(False)
ax.spines.bottom.set_visible(False)
# ax.set_yticks(size = 14)
# ax.set_xticks(size = 16)
ax.grid(True, axis = "y", color='w', linestyle='-', linewidth=1.2, which="major",  zorder = -1.0)
ax.patch.set_facecolor('.91')
plt.rc("axes", axisbelow = True)
# Create the boxplot

bp = ax.violinplot(data_to_plot, showmeans=False, showmedians=False, showextrema=True)
labels = ['Sham','Fixed Dose', 'Individualised dose']

for ax in [ax]:
    set_axis_style(ax, labels)

ax.set_xlabel('Stimulation condition', fontsize = 22, fontname = "sans")
ax.set_ylabel('Accuracy(%)', fontsize = 18, fontname = "sans")
ax.set_title('Accuracy', fontsize = 26, fontname = "sans")

# plt.rc('xtick', labelsize=30) 
# plt.rc('ytick', labelsize=15) 
# plt.rc('axes', titlesize=35)   
# plt.rc('axes', labelsize=35)    

for pc in bp['bodies']:
    pc.set_facecolor('#B2C8DF')
    pc.set_edgecolor('#6E85B7')
    pc.set_alpha(1)
quartile1, medians, quartile3 = np.percentile(data_to_plot, [25, 50, 75], axis=1)
whiskers = np.array([
    adjacent_values(sorted_array, q1, q3)
    for sorted_array, q1, q3 in zip(data_to_plot, quartile1, quartile3)])
whiskers_min, whiskers_max = whiskers[:, 0], whiskers[:, 0]

inds = np.arange(1, len(medians) + 1)
ax.scatter(inds, medians, marker='o', color='white', s=30, zorder=3)
ax.vlines(inds, quartile1, quartile3, color='k', linestyle='-', lw=5)
ax.vlines(inds, whiskers_min, whiskers_max, color='k', linestyle='-', lw=1)
# plt.savefig("/home/friction_15/lab/dissertation/reviewFigures/violinTacsAccuracy.jpg", format="jpg", dpi = 500)



dprime

In [ ]:
fix_dprime = dprime_dictt['Sham']['d_prime'].to_numpy()
sham_dprime = dprime_dictt['Fixed dose']['d_prime'].to_numpy()
ind_dprime = dprime_dictt['Individualised dose']['d_prime'].to_numpy()
# dprime_dict['sham'].tolist(), dprime_dict['fix'].tolist(), dprime_dict['ind'].tolist()

In [ ]:
data_to_plot = [sham_dprime, fix_dprime, ind_dprime ]
fig = plt.figure(figsize=(12, 7))
# Create an axes instance
ax = fig.add_axes([0.09,0.1,0.85,0.84])
ax.tick_params(axis='y', which='major', labelsize=12)
ax.spines.right.set_visible(False)
ax.spines.top.set_visible(False)
ax.spines.left.set_visible(False)
ax.spines.bottom.set_visible(False)
# ax.set_yticks(size = 14)
# ax.set_xticks(size = 16)
ax.grid(True, axis = "y", color='w', linestyle='-', linewidth=1.2, which="major",  zorder = -1.0)
ax.patch.set_facecolor('.91')
plt.rc("axes", axisbelow = True)
# Create the boxplot

bp = ax.violinplot(data_to_plot, showmeans=False, showmedians=False, showextrema=True)
labels = ['Sham','Fixed Dose', 'Individualised dose']
for ax in [ax]:
    set_axis_style(ax, labels)

ax.set_xlabel('Stimulation condition', fontsize = 22, fontname = "sans" )
ax.set_ylabel("d'", fontsize = 18, fontname = "sans")
ax.set_title("d'", fontsize = 26, fontname = "sans")

# plt.rc('xtick', labelsize=30)

# plt.rc('ytick', labelsize=15) 
# plt.rc('axes', titlesize=35)   
# plt.rc('axes', labelsize=35)  

for pc in bp['bodies']:
    pc.set_facecolor('#B2C8DF')
    pc.set_edgecolor('#6E85B7')
    pc.set_alpha(1)
quartile1, medians, quartile3 = np.percentile(data_to_plot, [25, 50, 75], axis=1)
whiskers = np.array([
    adjacent_values(sorted_array, q1, q3)
    for sorted_array, q1, q3 in zip(data_to_plot, quartile1, quartile3)])
whiskers_min, whiskers_max = whiskers[:, 0], whiskers[:, 0]

inds = np.arange(1, len(medians) + 1)
ax.scatter(inds, medians, marker='o', color='white', s=30, zorder=3)
ax.vlines(inds, quartile1, quartile3, color='k', linestyle='-', lw=5)
ax.vlines(inds, whiskers_min, whiskers_max, color='k', linestyle='-', lw=1)
# plt.savefig("/home/friction_15/lab/dissertation/reviewFigures/violinTacsDprime.jpg", format="jpg", dpi = 500)



In [ ]:
acc_fix = acc_dict["Fixed dose"]
acc_ind = acc_dict["Individualised dose"]

rt_fix = dataRt["Fixed dose"].to_list()
rt_ind = dataRt["Individualised dose"].to_list()

cond1 = ["Fixed"]*(len(acc_fix) + len(rt_fix))
cond2 = ["Individualised"]*(len(acc_ind) + len(rt_ind))



roi1 = ["Accuracy"]*len(acc_fix)
roi2 = ["RT"]*len(rt_fix)
roi3 = ["Accuracy"]*len(acc_ind)
roi4 = ["RT"]*len(rt_ind)

df_plot = pd.DataFrame({"data": acc_fix + rt_fix + acc_ind +  rt_ind, "cond" : cond1 + cond2, "ROI": roi1 + roi2 + roi3 + roi4})

In [ ]:
show_legend = [True,True,False,False]

fig = go.Figure()

i = 1 #i = 0 for RT; i = 1 for accuracy
fig.add_trace(go.Violin(x=df_plot['ROI'][(df_plot['cond'] == 'Fixed') &
                                (df_plot['ROI'] == pd.unique(df_plot['ROI'])[i])],
                    y=df_plot['data'][(df_plot['cond'] == 'Fixed')&
                                        (df_plot['ROI'] == pd.unique(df_plot['ROI'])[i])],
                    legendgroup='Fixed', scalegroup='Fixed', name='Fixed',
                    side='negative',
                    pointpos=-1.07, # where to position points
                    line_color='lightseagreen',
                    showlegend=show_legend[i],
                    xaxis = "x2"))
        

fig.add_trace(go.Violin(x=df_plot['ROI'][(df_plot['cond'] == 'Individualised') &
                                (df_plot['ROI'] == pd.unique(df_plot['ROI'])[i])],
                    y=df_plot['data'][(df_plot['cond'] == 'Individualised')&
                                        (df_plot['ROI'] == pd.unique(df_plot['ROI'])[i])],
                    legendgroup='Individualised', scalegroup='Individualised', name='Individualised',
                    side='positive',
                    pointpos=1.1, # where to position points
                    line_color='mediumpurple',
                    showlegend=show_legend[i],
                    xaxis="x2"
                    ))
                    


# update characteristics shared by all traces
fig.update_traces(meanline_visible=True,
                  points='all', # show all points
                  jitter=0.05,  # add some jitter on points for better visibility
                  scalemode='count') #scale violin plot area with total count
fig.update_layout(
    title_text="Behavioural Performance",
    xaxis_title = "ROI",
    yaxis_title = 'Accuracy (%)', #"Rt (ms)" if i =0 
    violingap=0.3, violingroupgap=0, violinmode='overlay',
    autosize = False,
    width = 700, 
    height = 500,
    font_size = 20 )
# fig.write_image("/home/friction_15/lab/dissertation/reviewFigures/acc_tACS.jpg", scale = 10)
fig.show()
# fig.write_image("E:\\rajat\python_plots\perform_acc.jpg", scale = 10)

In [ ]:
data_fix = pd.DataFrame()
data_ind = pd.DataFrame()
data_fix["dprime"] = dprime_dictt['Fixed dose']
data_fix["cond"] = ["Fixed"]*len(data_fix) 
data_ind["dprime"] = dprime_dictt['Individualised dose']
data_ind["cond"] = ["Individualised"]*len(data_ind)

df_dprime_violin = pd.concat([data_fix, data_ind])
df_dprime_violin["label"] = ["_"]*len(df_dprime_violin)
# xlabell = ["."]*len(dprime_dictt["Fixed dose"])
# df_dprime_violin = pd.DataFrame({"cond": cond_fix + cond_ind, "data": data_fix+data_ind, "label":xlabel+xlabell})
# df_dprime_violin.head()

In [ ]:
show_legend = [True,True,False,False]

fig = go.Figure()

# for i in range(0,len(pd.unique(df_plot['ROI']))):
i = 0
fig.add_trace(go.Violin(x= df_dprime_violin['label'][(df_dprime_violin['cond'] == 'Fixed') &
                                (df_dprime_violin['label'] == pd.unique(df_dprime_violin['label'])[i])],
                    y=df_dprime_violin['dprime'][(df_dprime_violin['cond'] == 'Fixed')&
                                        (df_dprime_violin['label'] == pd.unique(df_dprime_violin['label'])[i])],
                                        
                                        legendgroup='Fixed', scalegroup='Fixed', name='Fixed',
                    side='negative',
                    pointpos=-1.07, # where to position points
                    line_color='lightseagreen',
                    showlegend=show_legend[i],
                    ))
        

fig.add_trace(go.Violin(x=df_dprime_violin['label'][(df_dprime_violin['cond'] == 'Individualised') &
                                (df_dprime_violin['label'] == pd.unique(df_dprime_violin['label'])[i])],
                    y=df_dprime_violin['dprime'][(df_dprime_violin['cond'] == 'Individualised')&
                                        (df_dprime_violin['label'] == pd.unique(df_dprime_violin['label'])[i])],
                    legendgroup='Individualised', scalegroup='Individualised', name='Individualised',
                    side='positive',
                    pointpos=1.1, # where to position points
                    line_color='mediumpurple',
                    showlegend=show_legend[i],
                    
                    ))                    


# update characteristics shared by all traces
fig.update_traces(meanline_visible=True,
                  points='all', # show all points
                  jitter=0.05,  # add some jitter on points for better visibility
                  scalemode='count') #scale violin plot area with total count
fig.update_layout(
    title_text="Behavioural Performance",
    # xaxis_title = "ROI",
    yaxis_title = "d'",
    violingap=0.3, violingroupgap=0, violinmode='overlay',
    autosize = False,
    width = 700, 
    height = 500,
    font_size = 20 )
# fig.write_image("/home/friction_15/lab/dissertation/reviewFigures/dprime_tACS.jpg", scale = 10)
fig.show()
# fig.write_image("E:\\rajat\python_plots\perform_acc.jpg", scale = 10)